Copyright Hewlett Packard Enterprise Development LP.


# Convert OTF2 to Parquet with Timing
This notebook demonstrates how to convert OTF2 data to Parquet format while measuring the execution time for each step.

In [ ]:
# Import Required Libraries
import otf2
import pandas as pd
import hashlib
import time

In [52]:
# Define Helper Functions
def safe_dict(obj):
    if isinstance(obj, otf2.definitions.MetricClass):
        return {
            "members": [str(member) for member in obj.members]
        }
    elif hasattr(obj, '__dict__'):
        return {key: str(value) for key, value in vars(obj).items()}
    elif hasattr(obj, '_asdict'):
        return {key: str(value) for key, value in obj._asdict().items()}
    else:
        return {attr: str(getattr(obj, attr)) for attr in dir(obj) if not attr.startswith('_') and not callable(getattr(obj, attr))}

def generate_location_hash(location_name):
    return hashlib.md5(location_name.encode()).hexdigest()

In [ ]:
# Read OTF2 Data
def read_otf2_data(archive_name):
    start_time = time.time()
    data = {'events': [], 'definitions': []}
    with otf2.reader.open(archive_name) as trace:
        defs = trace.definitions
        eves = trace.events
        # Measure time taken to open the OTF2 archive
        open_end = time.time()
        print(f"Time taken to open OTF2 archive: {open_end - start_time:.2f} seconds")


        for def_type in dir(trace.definitions):
            if def_type.startswith('_') or def_type == 'attribute':
                continue
            definitions = getattr(trace.definitions, def_type)
            if callable(definitions):
                continue
            try:
                iterator = iter(definitions)
            except TypeError:
                data['definitions'].append(safe_dict(definitions))
            else:
                for definition in definitions:
                    data['definitions'].append(safe_dict(definition))

        dict_start_time = time.time()
        for location, event in trace.events:
            event_info = safe_dict(event)
            event_info['location'] = location.name
            event_info['location_hash'] = generate_location_hash(location.name)
            if isinstance(event, otf2.events.Enter):
                event_info['event'] = 'Enter'
                event_info['region'] = event.region.name
            elif isinstance(event, otf2.events.Leave):
                event_info['event'] = 'Leave'
                event_info['region'] = event.region.name
            data['events'].append(event_info)
        dict_end_time = time.time()
        print(f"Time taken to build dictionary for event: {dict_end_time - dict_start_time:.2f} seconds")

    end_time = time.time()
    print(f"Time taken to read OTF2 data: {end_time - start_time:.2f} seconds")
    return data

In [54]:
# Convert Data to Parquet
def convert_to_parquet(data, definitions_file, events_file):
    start_time = time.time()
    df_definitions = pd.DataFrame(data['definitions'])

    for col in df_definitions.select_dtypes(include=['object']).columns:
        df_definitions[col] = df_definitions[col].astype(str)

    df_definitions.to_parquet(definitions_file)
    # print("Definitions table:")
    # display(df_definitions)

    df_events = pd.DataFrame(data['events'])

    for col in df_events.select_dtypes(include=['object']).columns:
        df_events[col] = df_events[col].astype(str)

    df_events.to_parquet(events_file)
    # print("Events table:")
    # display(df_events)
    end_time = time.time()
    print(f"Time taken to convert data to Parquet: {end_time - start_time:.2f} seconds")

In [ ]:
# archive_name = "/workspace/scorep-traces/simple-mi300-example-run/traces.otf2"
archive_name = "/workspace/scorep-traces/frontier-hpl-run-using-2-ranks-with-craypm/traces.otf2"
definitions_file = "definitions.parquet"
events_file = "events.parquet"

start_time = time.time()
data = read_otf2_data(archive_name)

convert_to_parquet(data, definitions_file, events_file)
end_time = time.time()

print(f"Total execution time: {end_time - start_time:.2f} seconds")

Time taken to open OTF2 archive: 0.02 seconds
Time taken to build dictionary for event: 137.05 seconds
Time taken to read OTF2 data: 137.10 seconds
Time taken to convert data to Parquet: 110.80 seconds
Total execution time: 260.04 seconds


In [56]:
# Count and describe events
event_counts = {}
for event in data['events']:
    event_type = event.get('event', 'Unknown')
    event_counts[event_type] = event_counts.get(event_type, 0) + 1

total_events = len(data['events'])
print("\nEvent Summary:")
print(f"Total number of events: {total_events}")
print("Event types and their counts:")
for event_type, count in event_counts.items():
    print(f"  {event_type}: {count} events")


print("Unique locations:")
unique_locations = set(event.get('location', 'Unknown') for event in data['events'])
for location in unique_locations:
    print(f"  {location}")



Event Summary:
Total number of events: 16613424
Event types and their counts:
  Unknown: 2633870 events
  Enter: 6989777 events
  Leave: 6989777 events
Unique locations:
  
  HIP[0:1]
  HIP[3:0]
  HIP[1:2]
  HIP[2:0]
  HIP[3:1]
  HIP[3:2]
  HIP[0:3]
  HIP[2:1]
  Master thread
  Per process metrics
  HIP[0:2]
  HIP[2:3]
  HIP[2:2]
  HIP[1:0]
  HIP[1:3]
  HIP[0:0]
  HIP[1:1]
  HIP[3:3]


Timing from Reading craypm trace with default batch size (100):

```
Time taken to open OTF2 archive: 0.02 seconds
Time taken to build dictionary for event: 131.27 seconds
Time taken to read OTF2 data: 131.32 seconds

Event Summary:
Total number of events: 16613424
Event types and their counts:
  Unknown: 2633870 events
  Enter: 6989777 events
  Leave: 6989777 events

```

Timing with batch size equal to num events (16613424):

```
Time taken to open OTF2 archive: 0.02 seconds
Time taken to build dictionary for event: 156.49 seconds
Time taken to read OTF2 data: 156.54 seconds

Event Summary:
Total number of events: 16613424
Event types and their counts:
  Unknown: 2633870 events
  Enter: 6989777 events
  Leave: 6989777 events
```

Reading Cray PM with Batch size 16613:

```
Time taken to open OTF2 archive: 0.02 seconds
Time taken to build dictionary for event: 246.79 seconds
Time taken to read OTF2 data: 246.84 seconds

Event Summary:
Total number of events: 16613424
Event types and their counts:
  Unknown: 2633870 events
  Enter: 6989777 events
  Leave: 6989777 events
```